# FINE TUNING DISTILBERT (HUGGING FACE PRETRAINED TRANSFORMER) TO DETECT SQL INJECTIONS
### Rayen NAIT SLIMANE - Masters Student ISIMA INP

**NOTE:** This notebook is designed to run on [Kaggle](kaggle.com) with the [SQL-Injection-Dataset](https://www.kaggle.com/datasets/sajid576/sql-injection-dataset/data).
Make sure to enable the P100 GPU (free 30h/week quota) for optimal performance during training.

I propose fine-tuning DistilBERT, a high-efficiency transformer model, to deliver an AI-powered SQL Injection (SQLi) detection solution. This project will quickly evaluate the model's performance against a Kaggle dataset to establish the most effective and resource-lean strategy for eliminating SQLi vulnerabilities. The goal is to deploy a cutting-edge, high-accuracy defense faster than traditional methods allow.

## 1. Setup

In [ ]:
# Install required packages
%pip install transformers datasets torch evaluate scikit-learn tensorboard -q

In [ ]:
# Import libraries
import torch
import numpy as np
import pandas as pd
from datasets import load_dataset, Dataset
from transformers import (
    DistilBertTokenizer,
    DistilBertForSequenceClassification,
    TrainingArguments,
    Trainer
)
from sklearn.metrics import classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

In [5]:
# Check GPU availability
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"Number of GPUs: {torch.cuda.device_count()}")

PyTorch version: 2.6.0+cu124
CUDA available: True
CUDA device: Tesla P100-PCIE-16GB
Number of GPUs: 1


## 2. Data Loading

In [7]:
df = pd.read_csv('/kaggle/input/sql-injection-dataset/Modified_SQL_Dataset.csv')
df = df.rename(columns={'Query': 'text', 'Label': 'label'})

In [8]:
from sklearn.model_selection import train_test_split

# First split: 80% train+val, 20% test
train_val_df, test_df = train_test_split(
    df, test_size=0.2, random_state=42, stratify=df['label']
)

# Second split: 80% train, 20% validation (of the train_val set)
train_df, val_df = train_test_split(
    train_val_df, test_size=0.2, random_state=42, stratify=train_val_df['label']
)

In [9]:
from datasets import Dataset, DatasetDict

raw_dataset = DatasetDict({
    'train': Dataset.from_pandas(train_df.reset_index(drop=True)),
    'validation': Dataset.from_pandas(val_df.reset_index(drop=True)),
    'test': Dataset.from_pandas(test_df.reset_index(drop=True))
})

## 2. Tokenizer/Model Loading & Config

In [10]:
# Load the DistilBERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# Define preprocessing function
def preprocess_function(examples):
    """
    Tokenize the input text (SQL queries) with padding and truncation
    """
    return tokenizer(
        examples['text'], 
        padding="max_length", 
        truncation=True,
        max_length=512
    )

encoded_dataset = raw_dataset.map(preprocess_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Map:   0%|          | 0/19788 [00:00<?, ? examples/s]

Map:   0%|          | 0/4947 [00:00<?, ? examples/s]

Map:   0%|          | 0/6184 [00:00<?, ? examples/s]

In [11]:
# Load DistilBERT for sequence classification
# num_labels=2 for binary classification (benign vs malicious SQL queries)
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", 
    num_labels=2
)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [16]:
# Define training arguments - optimized for speed and performance (<10 min training)
training_args = TrainingArguments(
    output_dir="/kaggle/working/results",
    eval_strategy="epoch",                     # Evaluate once at end - faster
    save_strategy="epoch",
    learning_rate=3e-5,                        # Slightly higher - faster convergence
    per_device_train_batch_size=64,           # Larger batch - much faster
    per_device_eval_batch_size=128,           # Max out eval batch
    num_train_epochs=1,                        # Single pass is enough
    weight_decay=0.01,
    logging_steps=200,                         # Less frequent logging
    load_best_model_at_end=False,             # Skip loading - saves time
    save_total_limit=1,
    report_to="none",                          # Disable reporting overhead
    warmup_ratio=0.1,                          # 10% warmup - helps convergence
    fp16=True,                                 # Force mixed precision
    dataloader_num_workers=2,                 # Parallel data loading
    gradient_accumulation_steps=1,            # No accumulation needed with large batch
)

In [17]:
def compute_metrics(eval_pred):
    """
    Compute evaluation metrics
    """
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    # Calculate accuracy
    accuracy = (predictions == labels).mean()
    
    # Calculate precision, recall, and F1
    from sklearn.metrics import precision_recall_fscore_support
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average='binary'
    )
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['validation'],
    compute_metrics=compute_metrics
)

## 2. Model Fine Tuning & Evaluation

In [19]:
# Start training
train_result = trainer.train()

# Print training results
print("\nTraining completed!")
print(f"Training time: {train_result.metrics['train_runtime']:.2f} seconds")
print(f"Training samples per second: {train_result.metrics['train_samples_per_second']:.2f}")

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.092300,0.007534,0.997979,0.998898,0.995607,0.997250



Training completed!
Training time: 503.25 seconds
Training samples per second: 39.32


In [20]:
# Evaluate on validation set
eval_results = trainer.evaluate()
print("\nValidation Results:")
for key, value in eval_results.items():
    print(f"{key}: {value:.4f}")

# Get predictions on test set (if available)
if 'test' in encoded_dataset and len(encoded_dataset['test']) > 0:
    print("\nEvaluating on test set...")
    predictions = trainer.predict(encoded_dataset['test'])
    predicted_labels = np.argmax(predictions.predictions, axis=1)
    
    # Note: GLUE SST2 test set doesn't have labels
    # If using a custom dataset with test labels, you can compute metrics
    print("\nTest predictions generated!")

# Get predictions on validation set for detailed analysis
print("\nGenerating predictions for detailed analysis...")
val_predictions = trainer.predict(encoded_dataset['validation'])
val_predicted_labels = np.argmax(val_predictions.predictions, axis=1)
val_true_labels = encoded_dataset['validation']['label']

# Classification report
print("\nClassification Report:")
print(classification_report(
    val_true_labels, 
    val_predicted_labels,
    target_names=['Benign', 'Malicious']
))

# Confusion matrix
print("\nConfusion Matrix:")
cm = confusion_matrix(val_true_labels, val_predicted_labels)
print(cm)
print("\n[TN  FP]")
print("[FN  TP]")


Validation Results:
eval_loss: 0.0075
eval_accuracy: 0.9980
eval_precision: 0.9989
eval_recall: 0.9956
eval_f1: 0.9972
eval_runtime: 37.6194
eval_samples_per_second: 131.5010
eval_steps_per_second: 1.0370
epoch: 1.0000

Evaluating on test set...

Test predictions generated!

Generating predictions for detailed analysis...



Classification Report:
              precision    recall  f1-score   support

      Benign       1.00      1.00      1.00      3126
   Malicious       1.00      1.00      1.00      1821

    accuracy                           1.00      4947
   macro avg       1.00      1.00      1.00      4947
weighted avg       1.00      1.00      1.00      4947


Confusion Matrix:
[[3124    2]
 [   8 1813]]

[TN  FP]
[FN  TP]


In [21]:
# Load the saved model for inference
# Load from checkpoint
inference_model = DistilBertForSequenceClassification.from_pretrained("./results/checkpoint-310")
inference_tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# inference_model = DistilBertForSequenceClassification.from_pretrained("./fine_tuned_distilbert")
# inference_tokenizer = DistilBertTokenizer.from_pretrained("./fine_tuned_distilbert")

inference_model.to(device)
inference_model.eval()

# Test SQL queries
test_sentences = [
    "SELECT * FROM users WHERE username = 'admin'",
    "SELECT * FROM users WHERE id = 1 OR 1=1--",
    "SELECT name, email FROM customers WHERE id = 5",
    "'; DROP TABLE users; --",
    "SELECT * FROM products WHERE category = 'electronics' AND price < 100",
    "1' UNION SELECT null, table_name FROM information_schema.tables--"
]

print("\nTesting on SQL queries:")
for sentence in test_sentences:
    # Tokenize
    inputs = inference_tokenizer(
        sentence, 
        return_tensors="pt", 
        padding=True, 
        truncation=True, 
        max_length=512
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # Predict
    with torch.no_grad():
        outputs = inference_model(**inputs)
        prediction = torch.argmax(outputs.logits, dim=1).item()
        probabilities = torch.nn.functional.softmax(outputs.logits, dim=1)[0]
    
    sentiment = "Malicious" if prediction == 1 else "Benign"
    confidence = probabilities[prediction].item() * 100
    
    print(f"\nSQL Query: {sentence}")
    print(f"Classification: {sentiment} (Confidence: {confidence:.2f}%)")


Testing on SQL queries:

SQL Query: SELECT * FROM users WHERE username = 'admin'
Classification: Benign (Confidence: 99.88%)

SQL Query: SELECT * FROM users WHERE id = 1 OR 1=1--
Classification: Malicious (Confidence: 99.89%)

SQL Query: SELECT name, email FROM customers WHERE id = 5
Classification: Benign (Confidence: 99.82%)

SQL Query: '; DROP TABLE users; --
Classification: Malicious (Confidence: 99.88%)

SQL Query: SELECT * FROM products WHERE category = 'electronics' AND price < 100
Classification: Benign (Confidence: 99.89%)

SQL Query: 1' UNION SELECT null, table_name FROM information_schema.tables--
Classification: Malicious (Confidence: 99.89%)


In [22]:
# Save with clean name
model.save_pretrained("/kaggle/working/sqli_detector_model")
tokenizer.save_pretrained("/kaggle/working/sqli_detector_model")

('/kaggle/working/sqli_detector_model/tokenizer_config.json',
 '/kaggle/working/sqli_detector_model/special_tokens_map.json',
 '/kaggle/working/sqli_detector_model/vocab.txt',
 '/kaggle/working/sqli_detector_model/added_tokens.json')